In [76]:
import pandas as pd
import requests
from io import StringIO

pd.read_csv(
    StringIO(
        requests.get(
            "https://www.cs.cmu.edu/~biglou/resources/bad-words.txt"
        ).content.decode("utf-8")
    ),
    names=["word"],
).to_csv("outputs/bad-words.csv", index=False)

pd.concat(
    [
        pd.read_html(
            requests.get(f"https://hatebase.org/search_results/page={page_num}").content
        )[0]
        for page_num in range(1, 79)
    ]
).set_axis(["word", "language", "sighting", "offensive"], axis=1, inplace=False).to_csv(
    "outputs/hatebase.csv", index=False
)



In [77]:
import pandas as pd

hatebase = pd.read_csv("outputs/hatebase.csv")
hate_words = hatebase[hatebase["language"] == "English"]["word"]
hate_words = hate_words.str.split("(").str[0].str.strip()

toxic_words = pd.concat([hate_words, pd.read_csv("outputs/bad-words.csv")["word"],]).drop_duplicates()


In [78]:
import pandas as pd
from datasets import Dataset, DatasetDict

data_path = "./data/toxic_spans"
filename = "toxic_span_text_pairs.csv"

dataset = Dataset.from_pandas(pd.read_csv(data_path + "/" + filename))

dataset_dict = dataset.train_test_split(test_size=2 / 10, seed=42)
dataset_dict = DatasetDict(
    {
        "val": dataset_dict["test"],
        **dataset_dict["train"].train_test_split(test_size=3 / 10, seed=42),
    }
)
dataset_dict


DatasetDict({
    val: Dataset({
        features: ['original', 'censored'],
        num_rows: 3100
    })
    train: Dataset({
        features: ['original', 'censored'],
        num_rows: 8679
    })
    test: Dataset({
        features: ['original', 'censored'],
        num_rows: 3720
    })
})

In [79]:
import re


class SentenceDetoxer(object):
    def __init__(self, toxic_words):
        self.replacements = {toxic_word: "<CSD>" for toxic_word in self.toxic_words}
        self.pattern = "|".join(r"\b%s\b" % re.escape(s) for s in self.replacements)

    def __call__(self, sentence):
        return re.sub(
            self.pattern, lambda match: self.replacements[match.group(0)], sentence
        )


"To deny climate change is pretty much the equivalent of claiming a flat earth, or to believe that some zombified <CSD> will return to earth, or that aliens are among us.\n\nBecause we live in a society where 'everyone is equal' and the press is quick to 'ask the person on the street' their illiterate opinion we have a lot of completely <CSD> opinions passed around as valid - such as yours."